# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [3]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [4]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [5]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [6]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55564802, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [7]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [8]:
wv.most_similar('Texas')

[('Pennsylvania', 0.8023821711540222),
 ('Florida', 0.7974920272827148),
 ('Ohio', 0.7968348264694214),
 ('Louisiana', 0.7926130890846252),
 ('Arkansas', 0.79066002368927),
 ('Georgia', 0.7899303436279297),
 ('Maryland', 0.7872655391693115),
 ('Massachusetts', 0.7847886681556702),
 ('Oregon', 0.7837435007095337),
 ('Utah', 0.7821937799453735)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [18]:
wv.most_similar(negative=['Texas'])

[('on-campus', 0.4541440010070801),
 ('BlackMenSmiling', 0.4212934374809265),
 ('Butchering', 0.41403472423553467),
 ('evokes', 0.4121260344982147),
 ('Sprinkle', 0.40661436319351196),
 ('🎶Can', 0.392866849899292),
 ('savor', 0.38898998498916626),
 ('vaccine-laced', 0.38856953382492065),
 ('Expediting', 0.38782674074172974),
 ('non-judging', 0.38769102096557617)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [11]:
wv['Texas']

array([ 0.05496654, -1.6624634 , -1.5561576 , -1.7776964 ,  0.2829147 ,
        1.1349696 , -0.97775036, -0.583032  , -0.2155146 , -1.8847849 ,
       -0.34971556,  0.8462129 , -1.9844593 , -0.82165647, -0.67910504,
       -0.531007  , -1.4450047 , -1.0386821 ,  0.9963906 , -1.5401226 ,
        0.7905949 , -0.53316253,  0.67151684,  0.77666134, -1.6943158 ,
        0.42143157,  1.228319  ,  2.1314802 , -0.66461974,  0.1489874 ,
       -0.6966214 ,  1.4225829 ,  0.14932851,  1.8093499 , -0.3610302 ,
       -2.8690782 , -1.4484135 , -1.1654751 , -2.0987635 ,  0.42867744,
        1.3073518 ,  2.0428357 , -1.1970333 ,  0.99442756,  1.2383842 ,
       -1.0946206 ,  0.9063615 , -1.639243  ,  0.45942247,  2.7721245 ,
        2.1338735 ,  1.9236723 , -1.8934239 , -1.8341206 , -2.712069  ,
       -1.2718223 , -1.7260809 , -0.12018025, -0.01390992,  0.6210717 ,
       -1.0084888 , -1.905771  , -0.1319934 ,  0.64164466,  0.04115732,
        1.4119986 , -1.0295384 , -0.00414217, -0.23529488,  2.03

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [12]:
wv.vectors

array([[-0.33315822, -0.87866235,  0.05064983, ..., -0.11241678,
         0.72237754, -1.3198445 ],
       [ 1.6588916 ,  1.2733798 , -0.7179846 , ..., -0.15826261,
        -0.9117425 ,  0.25328237],
       [ 0.09786784,  0.9214569 , -0.7787878 , ..., -1.5882858 ,
        -0.38121384,  0.5858218 ],
       ...,
       [-0.093803  ,  0.10148206, -0.00344307, ...,  0.01972135,
         0.09408023,  0.04990669],
       [-0.02744038,  0.02472595, -0.03035314, ..., -0.02562218,
        -0.01025147, -0.02754463],
       [-0.06407076,  0.05053006,  0.02062516, ...,  0.09346881,
         0.11162516,  0.04625831]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [21]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6301048994064331),
 ('queen', 0.6243462562561035),
 ('title', 0.6098908185958862),
 ('brunette', 0.6010682582855225),
 ('crown', 0.5917714834213257),
 ('reminiscent', 0.5861700177192688),
 ('villain', 0.5792723894119263),
 ('centerpiece', 0.5737031698226929),
 ('revival', 0.572516918182373),
 ('symbol', 0.568787693977356)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!